<a href="https://colab.research.google.com/github/onobruno/Moodle_log/blob/master/Log_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Objetivo: Analisar a tabela log de forma ágil e simples para todos os cursos, independente do papel


In [0]:
### Bibliotecas
import pandas as pd
import matplotlib as matplot
import matplotlib.ticker as plticker
import matplotlib.pyplot as plt
import numpy as np
import pytz



from datetime import datetime 
from datetime import date
from google.colab import drive
from scipy import NaN
from datetime import datetime
from pathlib import Path as path



In [0]:
### load drive
drive.mount('/content/drive', force_remount=True)



In [0]:
class log():

    def __init__(self,file = None):
        print("...building the object...")
        self.path_file = file
        self.df = self.load_file()
        self.warning()
        self.ajustes()
        self.course = input('Qual é o nome do seu curso? ')

    def load_file(self):

        ### Here we will load the file with datastamp format###

        #transforming into a date format the first column with dates
        dateparse = lambda x: datetime.strptime(x,'%d/%m/%Y %H:%M')

        #Reading the file in appropriated way

        # Getting the suffix of the file
        file_suffix = path(self.path_file).suffix

        if file_suffix == '.xlsx':
            print ('The suffix file is ',file_suffix)
            df = pd.read_excel(self.path_file, parse_dates = [0] ,keep_date_col = True, date_parser = dateparse)
      
        elif file_suffix == '.xlx':
            print ('The suffix file is ',file_suffix)
            df = pd.read_excel(self.path_file, parse_dates = [0] ,keep_date_col = True, date_parser = dateparse)
      
        elif file_suffix == '.csv':
            print ('The suffix file is ',file_suffix)
            df = pd.read_csv(self.path_file, parse_dates = [0] ,keep_date_col = True, date_parser = dateparse)

        else:
            print('Your file is not using a extension at the end as: .xlsx, .xls or .csv. Please rename the file appropriated')

        return df

    def warning(self):

      if (self.df.shape[1]) < 9:
        print('This log table has ', 9 - self.df.shape[1] , 'columns less')

      elif (self.df.shape[1]) > 9:
        print('This log table has ', self.df.shape[1] - 9 , 'columns greater')

      else:
        print ('Load file susccesfully!')

    def ajustes(self):

      ### add a new column 'user_id' ###
      
      # getting user's id from description column at position 6
      users_id = [ int(line.split()[4].replace("'","")) for line in list(self.df.iloc[:,6]) ]
      self.df['user_id'] = users_id
      
      # spliting each line of 'Description' column
      self.df['description_split'] = self.df.iloc[:,6].str.split()

      # calcullating each line of 'Description' column
      length = [ len(line) for line in list(self.df.iloc[:,10])]
      self.df['description_lenth'] = length
      
    def visitas_por_dia(self):
      
      '''Número de diferentes alunos que visitaram a plataforma por dia'''

      #getting dataframe
      df_new = self.df

      #Without any modifications
      df_new = df_new[df_new.iloc[:,2]=='-']

      # Selecting just people, who saw the couse
      df_new = df_new[df_new.iloc[:,5]=='Curso visto']

      # reversing, indexing
      df_new = df_new.sort_index(ascending=False)
      df_new = df_new.reset_index(drop=True)
      df_new.index = df_new.iloc[:,0]
      
      # truncate
      inicial = input('Quando começa seu curso? (dd/mm/aaaa) \n')
      final = input('Quando termina seu curso? (dd/mm/aaaa) \n')

      inicial = datetime.strptime(inicial,"%d/%m/%Y") 
      final = datetime.strptime(final,"%d/%m/%Y") 

      #range between datas: data_inicial and data_final
      data_range  = pd.date_range(start = date_initial, end = date_final).to_list()
      df_new = df_new.truncate(after = pd.Timestamp(str(final.date()) + ' 23:59:00'), before=pd.Timestamp(str(inicial.date()) + ' 01:00:00'))

      #Getting just day,month and year
      df_new.iloc[:,0] = list(df_new.iloc[:,0].dt.date)

      #Getting only different days
      different_days = list(dict.fromkeys(list(df_new.iloc[:,0])))
      
      #Getting different user per day entered at main page course
      different_users_per_day = []

      for day in data_range:
        if day in different_days:
          df_day = df_new[df_new.iloc[:,0]==day]
          different_users = len(list(dict.fromkeys(list(df_day.iloc[:,9]))))
          different_users_per_day.append(different_users)
        else:
          different_users_per_day.append(0)
      
      #For plotting data
      different_days = [day.strftime('%d/%m') for day in data_range]
      df_plot =pd.DataFrame({'Dias': different_days, 'Usuários': different_users_per_day})

      # First plot
      plt.title("Acessos distintos por dia \n" + self.course)
      plt.ylabel('# Usuários')
      plt.xlabel('Data')
      plt.plot('Dias', 'Usuários', data=df_plot, marker='o', alpha=0.4)

      # Xticks      
      my_xticks = np.array(different_days)
      plt.xticks(my_xticks[::7])
      
      # Show the graph
      plt.show()

    def viewed_mod(self):

      '''This function will return how many times a user saw a module'''

      #getting dataframe
      df_new = self.df

      # Selecting just people, who saw modules
      df_new = df_new[df_new.iloc[:,5]=='Módulo do curso visualizado']
      
      ## example of message log: 
      ## "The user with id '102037' viewed the 'quiz' activity with course module id '99275'." -> this has 14 strings
      df_new = df_new[df_new.iloc[:,11] == 14]

      #Module id
      mod_id_list = [ mod_id[-1].replace("'","").replace(".","") for mod_id in list(df_new.iloc[:,10])]
      df_new['with id'] = ' with id '

      #module name it is at column 3 and its id
      mod_name_id_list = list(map(lambda x,y,z: x+y+z, list(df_new.iloc[:,3]), list(df_new['with id']), mod_id_list))
      df_new['mod_name_id'] = mod_name_id_list
            
      #Getting just the distincts modules's id with its name
      mods = list( dict.fromkeys( list(df_new['mod_name_id'] )) )
      mods.sort() #This is a inplace method

      #The id's user is being add with his name for distinct when there is equal names      
      id_name_list = [ item for item in zip( list(df_new['user_id']), list(df_new.iloc[:,1]))]
      df_new['id_name'] = id_name_list

      #Getting just the distincts user's id with his name
      users = list( dict.fromkeys( list(df_new['id_name']) ) )
      id_list,name_list =  map(list,zip(*users))
      
      # Initiating a dict with distinct modules as keys and inserting a empty list in each one
      mods_df = dict.fromkeys( mods ) 
      mods_df = {key:[] for key in mods_df.keys()}
    
      #Store user's infos
      user_df = []

      for user in users:

        user_df.append(user)
        df_new2 = df_new[df_new['id_name']== user]
        
        for mod in mods:
          if mod in list(df_new2['mod_name_id']):
              view_mod = list(df_new2['mod_name_id']).count(mod)
              mods_df[mod].append(view_mod)

          else:              
              mods_df[mod].append(0)
      
      #adjusting dataframe
      df_final = pd.DataFrame(mods_df)
      df_final.insert(0,'id_name',user_df)
      df_final.insert(0,'id',id_list)
      df_final.insert(0,'name',name_list)
      
      #ordering by name
      df_final = df_final.sort_values(['name'])
    
      #Saving dafaframe
      df_final.to_excel('Hits in ' + self.course + ' per module.xlsx',index=False)
    
    def time_spent(self):

      '''This function will return how long a user spent at the online course'''

      tempo = input('Quanto tempo dura a sessão do aluno em segundos? \n')
      tempo = int(tempo)

      #getting dataframe
      df_new = self.df

      #getting different users
      users = list(dict.fromkeys(list(df_new['user_id'])))
      
      all_tempos = []

      for user in users:

        df_new2 = df_new[df_new['user_id'] == user]
        lista = list(df_new2.iloc[:,0])

        if df_new2.shape[0] ==1:
          all_tempos.append(0)

        else:
          times = list( pd.to_datetime(df_new2.iloc[:,0]) )
          tempos = [ (time_after - time_before).total_seconds()/60 if (time_after - time_before).total_seconds() < tempo else 0 for time_after, time_before in zip(times[:-1],times[1:]) ]
          all_tempos.append(sum(tempos))
      
      d_times = {'Time spent (min)':all_tempos, 'User id': users}
      df_times = pd.DataFrame(d_times)
      df_times.to_excel('tempos' + self.course + 'users.xlsx')